# 1. Исследовательский анализ данных

In [2]:
pip install ipyplot

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import ipyplot
import numpy as np

In [5]:
labels = pd.read_csv('/datasets/faces/labels.csv')

In [ ]:
labels.info()

Размер выборки - 7591 фотография 

In [ ]:
labels.hist()

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
datagen_flow = datagen.flow_from_dataframe(
        dataframe=labels,
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=12345)

In [ ]:
features, target = next(datagen_flow)

In [ ]:
features

In [ ]:
features.shape

In [ ]:
target

In [ ]:
for i in features:
    plt.figure()
    plt.imshow(i)

In [ ]:
#ipyplot.plot_images(features,  max_images = 15, img_width = 150)

# 2. Обучение модели

Перенесите сюда код обучения модели и её результат вывода на экран.


(Код в этом разделе запускается в отдельном GPU-тренажёре, поэтому оформлен не как ячейка с кодом, а как код в текстовой ячейке)

In [ ]:
# обучение возрастного классификатора Resnet50
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import   Conv2D, Flatten, AvgPool2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator #загруузчик
from tensorflow.keras.applications.resnet import ResNet50 



def load_train(path):
    labels = pd.read_csv(path+'labels.csv')                                                     
    train_datagen = ImageDataGenerator(rescale= 1./255, validation_split=0.25)  
    train_datagen_flow = train_datagen.flow_from_dataframe(
        dataframe = labels,
        directory = path + 'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        subset='training', # установил subset='training', чтобы тест и трейн отличались
        seed=12345)

    return train_datagen_flow

def load_test(path):
    labels = pd.read_csv(path+'labels.csv')  
    test_datagen = ImageDataGenerator(rescale= 1./255, validation_split=0.25)  # Удалл validation split
    test_datagen_flow = test_datagen.flow_from_dataframe(
        dataframe = labels,
        directory = path + 'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        subset='validation', # установил subset='validation', чтобы тест и трейн отличались
        seed=12345)
    
    return test_datagen_flow

def create_model(input_shape):
    
    backbone = ResNet50(input_shape= input_shape,
                    weights='imagenet', 
                    include_top= False)  # без кол ва классов
                     
    # замораживаем ResNet50 без верхушки
    #backbone.trainable = False
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    #model.add(Flatten())                       # не было flatten
    model.add(Dense(1, activation='relu'))
    optimizer = Adam(lr=0.0005)
    model.compile(optimizer=optimizer, loss='mean_squared_error',metrics=['mae']) #пожно поменять на mean_absolute_error
    

    return model
    
    
def train_model(model, train_data, test_data, batch_size=None, epochs= 10,
                steps_per_epoch=None, validation_steps=None):
    
    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)
    model.fit(train_data, 
              validation_data= test_data,
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2) #shuffle=True   
    return model

# 3. Анализ обученной модели

Обученная модель показала ошибку в рамках допустимого диапозона. MAE составила 6.83.  Насколько я понял, можно было оставить 7 эпох, т.к. после 7ой она начала переобучаться. В качестве функции потерь использовалась MSE, т.к. говорят, что нейросети обучаются быстрей на ней, чем на MAE. 